# 需求分析
我们将构建一个可以与多种不同工具进行交互的Agent：一个是本地数据库，另一个是搜索引擎。你能
够向该Agent提问，观察它调用工具，并与它进行对话。

**涉及的功能：**
* 使用语言模型，特别是它们的工具调用能力
* 创建检索器以向我们的Agent公开特定信息
* 使用搜索工具在线查找信息
* 提供聊天历史，允许聊天机器人“记住”不同id 过去的交互，并在回答后续问题时考虑它们。

# 代码实现
### 定义工具

In [2]:
import os, dotenv
from langchain_community.tools.tavily_search import TavilySearchResults

dotenv.load_dotenv()
# 定义 AVILY_KEY 密钥
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
# 查询 Tavily 搜索 API
search = TavilySearchResults(max_results=1)
# 执行查询
res = search.invoke("今天上海天气怎么样")
print(res)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_29316\338906290.py:8: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search = TavilySearchResults(max_results=1)


[{'title': '上海 - 中国气象局-天气预报-城市预报', 'url': 'https://weather.cma.cn/web/weather/58367.html', 'content': '|  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |\n| 气温 | 14.5℃ | 17.3℃ | 18℃ | 16.1℃ | 14.2℃ | 13.9℃ | 13.5℃ | 13.5℃ |\n| 降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 |\n| 风速 | 7.9m/s | 7.9m/s | 7.8m/s | 7.4m/s | 7.3m/s | 7.8m/s | 7.9m/s | 7.8m/s |\n| 风向 | 西北风 | 东北风 | 东北风 | 东北风 | 西北风 | 无持续风向 | 东北风 | 东北风 |\n| 气压 | 1029hPa | 1029.1hPa | 1027.4hPa | 1027.2hPa | 1027.2hPa | 1026.8hPa | 1025.5hPa | 1024.7hPa | [...] |  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |\n| 气温 | 19.5℃ | 20.2℃ | 22.6℃ | 21.6℃ | 18.6℃ | 18.3℃ | 18℃ | 16.5℃ |\n| 降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 |\n| 风速 | 7.9m/s | 7.9m/s | 7.8m/s | 7.7m/s | 7.8m/s | 7.9m/s | 7.9m/s | 7.9m/s |\n| 风向 | 东北风 | 东南风 | 东南风 | 东南风 | 东南风 | 东南风 | 东南风 | 东南风 |\n

## 定义Retriever
Retriever 是 langchain 库中的一个模块，用于检索工具。
根据上述查询结果中的某个URL中，获取一些数据创建一个检索器。

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import dotenv

dotenv.load_dotenv()
# 1. 提供一个大模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
embedding_model = OpenAIEmbeddings()
# 2.加载HTML内容为一个文档对象
loader = WebBaseLoader("https://zh.wikipedia.org/wiki/%E7%8C%AB")
docs = loader.load()
#print(docs)
# 3.分割文档
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
documents = splitter.split_documents(docs)
# 4.向量化 得到向量数据库对象
vector = FAISS.from_documents(documents, embedding_model)
# 5.创建检索器
retriever = vector.as_retriever()
# 测试检索结果
print(retriever.invoke("猫的特征")[0])

USER_AGENT environment variable not set, consider setting it to identify your requests.
D:\develop\miniconda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


page_content='感官[编辑]
貓的感官適於狩獵。在哺乳類動物中，貓的聽覺、視覺、嗅覺、味覺、觸覺極敏銳。

視覺[编辑]
貓的瞳孔能縮得如線般狹小
貓的瞳孔變化
貓在晝間視覺縱不及人類，夜視能力與追蹤視覺上之活動物件卻相當出色，夜視能力是人類的六倍，雖然綜合色彩計算整體視覺系數則僅及人類的十分之一。貓的眼睛具有微光觀察能力，即使只有微弱月光都可分辨物件，有關光線入貓眼後可放大40至50倍處理，令貓具有夜間活動的能力，即使在黑暗的地下室貓咪依然能活動自如。強光下，貓會將瞳孔縮得如線般狹小，以減少對視網膜的傷害，但視野會因而縮窄。由於貓眼具備高幀與高分辨率視覺，故此電視機極微細之動靜對貓而言亦成逐格跳躍之畫面。[51]
另外，不只是光線會影響貓的瞳孔，感情也會。一般而言貓放鬆的時候瞳孔會縮小，而緊張時會放大。
貓的視網膜背面有一層藍綠色如熒光一般的薄膜（Tapetum Lucidum），可增加在暗處的視力。閃光中，貓眼能呈現各式各樣顔色。如同多數食肉動物，貓眼長在臉上朝正前方，賦予其遼闊的視野，單一貓眼視野為100度，雙眼視野為285度，加上頸旋靈活，總計視野比人類雙眼僅100度靈活得多。不過，貓僅能聚焦其前30厘米至3米之物件，相對人類而言屬大近視。
貓對三原色的辨識力很差，相對人類而言，貓有黃藍色盲且紅綠色辨成灰黃色，與狗相似。關於貓的夜視能力，生物學家發現牛磺酸對貓的視力起了很大作用，貓本身不能合成牛磺酸，必須由外攝取。缺乏牛磺酸，會使視力及夜視能力變差，影響夜間活動。據生物學家稱，貓經常捕鼠，是因老鼠體內含牛磺酸。[52][53]
紫外線可以通過貓的晶状体。
當四周光線微弱，貓會用感覺毛來改善行動力與感知能力。感覺毛主要分布於鼻子兩側、下巴、雙眼上方、兩頰也有數根。感覺毛可感受非常微弱的空氣波動，視野不清時也能協助辨識阻礙位置。鬍鬚尖端與雙耳連成一線，恰是身體能通過障礙的最小範圍，故貓可在黑夜中快速判斷地形能否通過。
貓有第三眼瞼，當貓眼睑張開時，眨眼時第三眼瞼會從旁稍微遮蓋眼睛。若貓生病，或是睡眠，笑著，此眼皮會縮回一部分。若貓長時間嶄露第三眼瞼，表示它的健康有問題。' metadata={'source': 'https://zh.wikipedia.org/wiki/%E7%8C%AB', 'title': '猫 - 维基百科，自由的百科全书'

## 创建工具、工具集
将上一步定义的retriever包装为一个工具tool

In [4]:
from langchain.tools.retriever import create_retriever_tool

# 创建一个工具来检索文档
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="wiki_search",
    description="搜索维基百科",
)

创建工具集：

In [6]:
tools = [search, retriever_tool]

## 语言模型调用工具

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 获取大模型
model = ChatOpenAI(model="gpt-4o-mini")
# 模型绑定工具
model_with_tools = model.bind_tools(tools)
# 根据输入自动调用工具
messages = [HumanMessage(content="今天上海天气怎么样")]
response = model_with_tools.invoke(messages)
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': '上海天气'}, 'id': 'call_FnDR3cvFi0WGs4nNyG3JVTQS', 'type': 'tool_call'}]


我们可以看到现在没有内容，但有一个工具调用！它要求我们调用Tavily Search工具。

这并不是在调用该工具，只是告诉我们要调用。为了实际调用它，我们将创建我们的Agent程序。

## 创建Agent程序(使用通用方式)
以FUNCTION CALL方式进行调用：

In [8]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
print(prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad')]


现在，我们可以使用LLM、提示和工具初始化Agent。Agent 负责接收输入并决定采取什么行动。关键
的是，Agent 不执行这些操作 - 这是由AgentExecutor（下一步）完成的。

In [10]:
from langchain_classic.agents import create_tool_calling_agent
from langchain_classic.agents import AgentExecutor

# 创建Agent对象
agent = create_tool_calling_agent(model, tools, prompt)
# 创建AgentExecutor对象
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## 运行Agent
现在我们运行Agent程序，看agent_executor是如何调用工具的。

In [12]:
print(agent_executor.invoke({"input": "猫的特征"}))



> Entering new AgentExecutor chain...

Invoking: `wiki_search` with `{'query': '猫'}`


一隻貓正在吃碗中的乾猫粮
行為[编辑]
一只望向窗外的猫
排泄[编辑]
貓在排泄後會掩埋其排泄物。在中国形容人做事马虎的谚语“猫儿埋狗屎”。這是一種為了掩藏自己氣息的做法，因為在野外貓並不是很高級的獵食者，需要把自己藏起來不被天敵發現。但是一些家貓生活在比較安全的環境裡，會漸漸放棄掩埋自己的排泄物，這種變化有可能是後天形成的。

睡眠[编辑]
貓為補充精力，睡眠時間比較其它動物長久。每日睡眠時間在12至16小時間，平均在13至14小時，部分更會睡眠20小時，惟只有30%的睡眠為酣睡期，其餘睡覺時間皆為非酣睡期，容易驚醒。在英文中，“cat nap”（打盹）即為指能夠像貓一樣快速入睡。牠們常在晚間活動，但並不表示他們是夜行性動物。

發聲[编辑]




喵


播放此文件有问题？请参见媒體幫助。

参见：喵
各個語言對貓叫聲（ⓘ）的擬聲詞，在漢語為喵（miāo）、韓語為yang-yang（양양）[來源請求]、日語為nyaa（にゃあ）、美式英語為meow、英式英語為miaow、法語為miaou，以及其他不同的語言拼法。貓叫聲有著其意味深長的意涵，貓也會發出使人舒服[來源請求]，開心的呜呜聲，和低聲嘶吼，貓甚至可以同時喵喵、貓貓叫或發出呜呜聲，儘管這很少見。有時貓也會嗥叫或鳴叫。除了喵喵叫以外，貓還會發出咕嚕聲（ⓘ），推測可能是延續小貓時想喝奶時所發出的聲音，不確定如何發出這種特有的咕嚕聲，可能從喉嚨裡、聲帶附近的薄膜振動所產生，成年貓發出咕嚕聲通常表示友好、滿意、放鬆，但生病或煩躁的貓偶爾也會發出咕嚕聲。猫在遇到敌人时，也会发出像蛇一样的嘶嘶声，用来震慑敌人。另外貓發現到遠方的鳥類時，會非常專注的凝視並發出『啊啊啊啊』、『嘎嘎嘎嘎』的高頻叫聲，英语中稱為貓鳴（cat chirping）或貓啁啾（cat chattering），飼養在室內或大樓裡的貓尤常出現這種狀況，對著窗外的鳥發出這種叫聲。某些貓先天無法發聲，稱為無聲貓，亦有聲帶被割而無法發聲者。

遊戲[编辑]
幼貓常會藉與其他貓遊戲以訓練自己捕捉獵物，由人類豢養的貓大多會用人造的逗貓棒予貓玩。就室內貓而言，遊戲為良好運動，亦能夠增進與同伴

In [13]:
result = agent_executor.invoke({"input": "2025年11月8号，宁波江北区的天气如何"})
print(result)



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '2025年11月8日 宁波江北区天气预报'}`


[{'title': '预报- 宁波 - 中国天气网', 'url': 'https://www.weather.com.cn/weather15d/101210401.shtml', 'content': '周二（11日） 阴 18℃/14℃ 北风 <3级\n 周三（12日） 多云转雨 18℃/14℃ 东北风转北风 <3级\n 周四（13日） 雨 18℃/13℃ 北风转西北风 3-4级\n 周五（14日） 雨 16℃/13℃ 西北风转北风 3-4级转<3级\n 周六（15日） 晴转多云 18℃/11℃ 北风转西北风 3-4级转<3级\n 周日（16日） 阴 19℃/12℃ 南风 <3级\n 周一（17日） 阴 22℃/13℃ 东北风转南风 <3级\n 周二（18日） 阴转晴 21℃/16℃ 南风 <3级\n\n8-15天预报属于客观预报产品，反映的是未来天气变化趋势、请随时关注最新预报.....\n\n8-15天预报，是集合多家全球数值天气预报模式客观预报产品加工而成，未经预报员主观订正，反映未来一段时间内天气变化趋势，具有一定的不确定性，供公众参考，欲知更加准确的天气预报需随时关注短期天气预报和最新预报信息更新。\n\n# 天气资讯\n\n:   俯瞰江西遂川农田 田间蔬菜管护忙  中国天气网 2025-11-04 10:38', 'score': 0.73138535}]关于2025年11月8日宁波江北区的天气预报，目前的信息显示该日期的具体天气尚未完全可知。最新的天气预测主要为8到15天的趋势，包含了一些未来几天气象状况的初步信息。

对于具体的天气情况，建议您在接近日期时查看最新的天气预报，以获得更加准确的信息。

您可以访问 [中国天气网](https://www.weather.com.cn/weather15d/101210401.shtml) 来获取最新的天气预报更新。

> Finished chain.
{'input': '2025年11月8号，宁波江北区的天气如何', 'output': '关于20

## 添加记忆
设计每一个用户有单独的记忆存储

In [14]:
from langchain_community.chat_message_histories import ChatMessageHistory

from langchain_core.chat_history import BaseChatMessageHistory

from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

# 调取指定session_id对应的memory
def get_session_history(session_id: str) -> BaseChatMessageHistory:

    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]

agent_with_chat_history = RunnableWithMessageHistory(
    runnable=agent_executor,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

response = agent_with_chat_history.invoke(
    {"input": "Hi，我的名字是Cyber"},
    config={"configurable": {"session_id": "123"}},
)

print(response)



> Entering new AgentExecutor chain...
你好，Cyber！很高兴见到你！有什么我可以帮助你的吗？

> Finished chain.
{'input': 'Hi，我的名字是Cyber', 'chat_history': [], 'output': '你好，Cyber！很高兴见到你！有什么我可以帮助你的吗？'}


再问一次：

In [15]:
response = agent_with_chat_history.invoke(
    {"input": "你还知道我是谁吗？"},
    config={"configurable": {"session_id": "123"}},
)

print(response)



> Entering new AgentExecutor chain...
抱歉，我没有关于你的个人信息，除了你告诉我的名字。如果你愿意的话，可以告诉我更多关于你的信息，或者问我任何你想知道的问题！

> Finished chain.
{'input': '你还知道我是谁吗？', 'chat_history': [HumanMessage(content='Hi，我的名字是Cyber', additional_kwargs={}, response_metadata={}), AIMessage(content='你好，Cyber！很高兴见到你！有什么我可以帮助你的吗？', additional_kwargs={}, response_metadata={})], 'output': '抱歉，我没有关于你的个人信息，除了你告诉我的名字。如果你愿意的话，可以告诉我更多关于你的信息，或者问我任何你想知道的问题！'}
